In [1]:
# TODO check right order in DEG

In [1]:
import functools

import jax
import numpy as np
import scanpy as sc

import cfp.preprocessing as cfpp
from cfp.metrics import compute_metrics_fast

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
i=5
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_train_{i}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_test_{i}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/adata_ood_{i}.h5ad"

In [3]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)
#adata_pred_test = sc.read("/lustre/groups/ml01/workspace/ot_perturbation/models/chemcpa/sciplex/adata_biolord_test_predictions.h5ad")
#adata_pred_ood = sc.read("/lustre/groups/ml01/workspace/ot_perturbation/models/chemcpa/sciplex/adata_biolord_ood_predictions.h5ad")

In [4]:
adata_preds_ood = sc.read_h5ad(f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/chemcpa/adata_chemcpa_split_{i}_pred_ood.h5ad")
adata_preds_test = sc.read_h5ad(f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/chemcpa/adata_chemcpa_split_{i}_pred_test.h5ad")




In [5]:
adata_preds_ood.n_obs, adata_preds_test.n_obs

(104673, 2147100)

In [6]:
adata_preds_ood

AnnData object with n_obs × n_vars = 104673 × 2001
    obs: 'cell_line', 'condition', 'perturbation', 'dose', 'pert_category'

In [7]:
adata_preds_ood.obs["dose_adapted"] = adata_preds_ood.obs.apply(lambda x: 0.0 if x["dose"]==1.0 else x["dose"], axis=1)
adata_preds_ood.obs["condition"] = adata_preds_ood.obs.apply(lambda x: x["cell_line"]+ "_"+x["perturbation"]+"_"+str(x["dose_adapted"]), axis=1)

adata_preds_test.obs["dose_adapted"] = adata_preds_test.obs.apply(lambda x: 0.0 if x["dose"]==1.0 else x["dose"], axis=1)
adata_preds_test.obs["condition"] = adata_preds_test.obs.apply(lambda x: x["cell_line"]+ "_"+x["perturbation"]+"_"+str(x["dose_adapted"]), axis=1)

In [8]:
conditions_ood = set(adata_ood.obs["condition"].unique())
conditions_ood_pred = set(adata_preds_ood.obs["condition"].unique())

conditions_test = set(adata_test.obs["condition"].unique())
conditions_test_pred = set(adata_preds_test.obs["condition"].unique())

In [9]:
set(adata_preds_ood.obs["condition"].unique())==set(adata_preds_test.obs["condition"].unique()) 

False

In [10]:
conditions_ood == conditions_ood_pred

True

In [11]:
conditions_test == conditions_test_pred

True

In [12]:
conditions_ood_pred == conditions_test_pred

False

In [56]:
len(conditions_test_pred), len(conditions_test), len(conditions_test_pred.intersection(conditions_test))

(86, 2045, 3)

In [50]:
len(conditions_ood_pred)

86

In [ ]:
conditi

In [49]:
conditions_test_pred

{'A549_Alvespimycin_(17-DMAG)_HCl_10.0',
 'A549_Alvespimycin_(17-DMAG)_HCl_100.0',
 'A549_Belinostat_(PXD101)_10.0',
 'A549_Belinostat_(PXD101)_100.0',
 'A549_Belinostat_(PXD101)_1000.0',
 'A549_Dacinostat_(LAQ824)_10.0',
 'A549_Dacinostat_(LAQ824)_100.0',
 'A549_Dacinostat_(LAQ824)_1000.0',
 'A549_Flavopiridol_HCl_10.0',
 'A549_Flavopiridol_HCl_100.0',
 'A549_Flavopiridol_HCl_1000.0',
 'A549_Flavopiridol_HCl_10000.0',
 'A549_Givinostat_(ITF2357)_10.0',
 'A549_Givinostat_(ITF2357)_100.0',
 'A549_Givinostat_(ITF2357)_1000.0',
 'A549_Givinostat_(ITF2357)_10000.0',
 'A549_Hesperadin_10.0',
 'A549_Hesperadin_100.0',
 'A549_Hesperadin_1000.0',
 'A549_Quisinostat_(JNJ-26481585)_2HCl_10.0',
 'A549_Quisinostat_(JNJ-26481585)_2HCl_100.0',
 'A549_TAK-901_10.0',
 'A549_TAK-901_100.0',
 'A549_TAK-901_1000.0',
 'A549_Tanespimycin_(17-AAG)_10.0',
 'A549_Tanespimycin_(17-AAG)_100.0',
 'A549_Tanespimycin_(17-AAG)_1000.0',
 'A549_Vehicle_0.0',
 'K562_Alvespimycin_(17-DMAG)_HCl_10.0',
 'K562_Alvespimyci

In [32]:
ood_split = i
adata_pred_test = sc.read_h5ad(f"/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/chemcpa/adata_chemcpa_split_{ood_split}_pred_test.h5ad")


In [34]:
adata_pred_test.obs.head()

,cell_line,condition,perturbation,dose,pert_category
0,MCF7,control,Vehicle,1.0,NaN
1,MCF7,control,Vehicle,1.0,NaN
2,MCF7,control,Vehicle,1.0,NaN
3,MCF7,control,Vehicle,1.0,NaN
4,MCF7,control,Vehicle,1.0,NaN


In [9]:
adata_preds_ood.obs["dose"].value_counts()

dose
10.0       33614
100.0      31330
1000.0     25175
10000.0    11554
1.0         3000
Name: count, dtype: int64

In [11]:
adata_preds_ood.obs.head()

,cell_line,condition,perturbation,dose,pert_category
0,MCF7,control,Vehicle,1.0,NaN
1,MCF7,control,Vehicle,1.0,NaN
2,MCF7,control,Vehicle,1.0,NaN
3,MCF7,control,Vehicle,1.0,NaN
4,MCF7,control,Vehicle,1.0,NaN


In [12]:
adata_preds_ood.obs["perturbation"].value_counts()

perturbation
Givinostat_(ITF2357)               14027
Belinostat_(PXD101)                12864
Hesperadin                         12846
Tanespimycin_(17-AAG)              12025
Dacinostat_(LAQ824)                11253
TAK-901                            10836
Alvespimycin_(17-DMAG)_HCl          9568
Flavopiridol_HCl                    9331
Quisinostat_(JNJ-26481585)_2HCl     8923
Vehicle                             3000
Name: count, dtype: int64

In [18]:
adata_preds_ood.obs["dose_adapted"] = adata_preds_ood.obs.apply(lambda x: 0.0 if x["dose"]==1.0 else x["dose"], axis=1)
adata_preds_ood.obs["condition"] = adata_preds_ood.obs.apply(lambda x: x["cell_line"]+ "_"+x["perturbation"]+"_"+str(x["dose_adapted"]), axis=1)

In [26]:
adata_pred_ood = adata_preds_ood

In [27]:
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if "Vehicle" in cond:
        continue
    src_str = list(adata_ood[adata_ood.obs["condition"] == cond].obs["cell_line"].unique())
    assert len(src_str) == 1
    pred = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].X
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    
    ood_data_target_decoded_predicted[cond] = pred
    

In [5]:
adata_preds.obs["pert_category"].value_counts()

pert_category
MCF7_Fasudil_(HA-1077)_HCl_10000.0           1589
MCF7_Tacedinaline_(CI994)_10.0               1575
MCF7_Tofacitinib_(CP-690550)_Citrate_10.0    1566
MCF7_Gandotinib_(LY2784544)_10.0             1538
MCF7_Tranylcypromine_(2-PCPA)_HCl_10.0       1533
                                             ... 
K562_TAK-901_1000.0                          1109
MCF7_Regorafenib_(BAY_73-4506)_10000.0       1109
A549_Epothilone_A_10000.0                    1109
K562_Nilotinib_(AMN-107)_1000.0              1106
A549_Roxadustat_(FG-4592)_10.0               1103
Name: count, Length: 522, dtype: int64

In [6]:
adata_preds.obs["condition_new"] = adata_preds.obs["pert_category"]

In [7]:
adata_pred_ood = adata_preds

In [66]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [68]:
adata_preds

NameError: name 'adata_preds' is not defined

In [69]:
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood)
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)

cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_ood)
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)

In [70]:
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if "Vehicle" in cond:
        continue
    src_str = list(adata_test[adata_test.obs["condition"] == cond].obs["cell_type"].unique())
    assert len(src_str) == 1
    pred = adata_pred_test[adata_pred_test.obs["condition"] == cond].X

    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = pred
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if "Vehicle" in cond:
        continue
    src_str = list(adata_ood[adata_ood.obs["condition"] == cond].obs["cell_type"].unique())
    assert len(src_str) == 1
    pred = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].X
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    
    ood_data_target_decoded_predicted[cond] = pred
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

In [12]:
#test_deg_dict = {
#    k: v
#    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
#    if k in test_data_target_decoded_predicted.keys()
#}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [13]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

#test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
#test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [14]:
#test_metrics_encoded = jax.tree_util.tree_map(compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted)
#mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

#test_metrics_decoded = jax.tree_util.tree_map(compute_metrics, test_data_target_decoded, test_data_target_decoded_predicted)
#mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [28]:
from cfp.metrics import compute_metrics_fast, compute_mean_metrics, compute_metrics

In [29]:
#ood_metrics_encoded = jax.tree_util.tree_map(
#    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
#)
#mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [30]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.8906210459858538,
 'decoded_ood_e_distance': 7.785849027126996,
 'decoded_ood_mmd_distance': 0.33446480944214096}

In [72]:
from sklearn.metrics import r2_score
ood_metrics_decoded = jax.tree_util.tree_map(
    r2_score, ood_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_ood_")

ValueError: Dict key mismatch; expected keys: ['A549_Alvespimycin_(17-DMAG)_HCl_10.0', 'A549_Alvespimycin_(17-DMAG)_HCl_100.0', 'A549_Belinostat_(PXD101)_10.0', 'A549_Belinostat_(PXD101)_100.0', 'A549_Belinostat_(PXD101)_1000.0', 'A549_Dacinostat_(LAQ824)_10.0', 'A549_Dacinostat_(LAQ824)_100.0', 'A549_Dacinostat_(LAQ824)_1000.0', 'A549_Flavopiridol_HCl_10.0', 'A549_Flavopiridol_HCl_100.0', 'A549_Flavopiridol_HCl_1000.0', 'A549_Flavopiridol_HCl_10000.0', 'A549_Givinostat_(ITF2357)_10.0', 'A549_Givinostat_(ITF2357)_100.0', 'A549_Givinostat_(ITF2357)_1000.0', 'A549_Givinostat_(ITF2357)_10000.0', 'A549_Hesperadin_10.0', 'A549_Hesperadin_100.0', 'A549_Hesperadin_1000.0', 'A549_Quisinostat_(JNJ-26481585)_2HCl_10.0', 'A549_Quisinostat_(JNJ-26481585)_2HCl_100.0', 'A549_TAK-901_10.0', 'A549_TAK-901_100.0', 'A549_TAK-901_1000.0', 'A549_Tanespimycin_(17-AAG)_10.0', 'A549_Tanespimycin_(17-AAG)_100.0', 'A549_Tanespimycin_(17-AAG)_1000.0', 'K562_Alvespimycin_(17-DMAG)_HCl_10.0', 'K562_Alvespimycin_(

In [73]:
mean_test_metrics_decoded

NameError: name 'mean_test_metrics_decoded' is not defined

In [35]:
# TODO CHECK THAT ALL CONDITIONS ARE THERE

In [20]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

#deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
#deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


2024-09-26 17:39:02.393374: E external/xla/xla/service/hlo_lexer.cc:438] Failed to parse int literal: 28675072374034695991782
2024-09-26 17:39:05.159321: E external/xla/xla/service/hlo_lexer.cc:438] Failed to parse int literal: 28675072374034695991782


In [18]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9736756702949261,
 'decoded_ood_e_distance': 1.710632515007245,
 'decoded_ood_mmd_distance': 0.3092773672321747}

In [ ]:
deg_mean_ood_metrics

In [22]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.8980501904082919,
 'decoded_ood_sinkhorn_div_1': 24.087488019788587,
 'decoded_ood_sinkhorn_div_10': 10.629761876286688,
 'decoded_ood_sinkhorn_div_100': 2.7574647439492717,
 'decoded_ood_e_distance': 5.1997032814852515,
 'decoded_ood_mmd': 0.21532548593854323}

In [24]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.7707121466237041,
 'deg_ood_sinkhorn_div_1': 4.967486492484002,
 'deg_ood_sinkhorn_div_10': 2.5098944519822664,
 'deg_ood_sinkhorn_div_100': 2.1683613976111284,
 'deg_ood_e_distance': 4.266040287749948,
 'deg_ood_mmd': 0.19586701085056843}

In [20]:
mean_test_metrics_encoded

{'encoded_test_r_squared': 0.9663768763027192,
 'encoded_test_sinkhorn_div_1': 5.592183408591178,
 'encoded_test_sinkhorn_div_10': 0.9784451699041209,
 'encoded_test_sinkhorn_div_100': 0.29335878487668604,
 'encoded_test_e_distance': 0.4529303384860695,
 'encoded_test_mmd': 0.30223043528903787}

In [21]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.9776814478837761,
 'decoded_test_sinkhorn_div_1': 35.21613923791555,
 'decoded_test_sinkhorn_div_10': 20.111658681658465,
 'decoded_test_sinkhorn_div_100': 1.0817417683664428,
 'decoded_test_e_distance': 1.142540392218624,
 'decoded_test_mmd': 0.37149997041274985}

In [22]:
1

1

In [35]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9217317916065486,
 'decoded_ood_sinkhorn_div_1': 23.054601695086504,
 'decoded_ood_sinkhorn_div_10': 9.644866092784985,
 'decoded_ood_sinkhorn_div_100': 1.9331934129869617,
 'decoded_ood_e_distance': 3.578220447385841,
 'decoded_ood_mmd': 0.2573978977324021}

In [36]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': 0.8452027128954586,
 'encoded_ood_sinkhorn_div_1': 4.813160806088834,
 'encoded_ood_sinkhorn_div_10': 1.7860163869084538,
 'encoded_ood_sinkhorn_div_100': 1.4259318918795199,
 'encoded_ood_e_distance': 2.7788584662605174,
 'encoded_ood_mmd': 0.22473051721413326}

In [39]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.8268589686731178,
 'deg_ood_sinkhorn_div_1': 4.0035812242611035,
 'deg_ood_sinkhorn_div_10': 1.6815547105428335,
 'deg_ood_sinkhorn_div_100': 1.44983539146346,
 'deg_ood_e_distance': 2.855253432602434,
 'deg_ood_mmd': 0.21982185384478872}

In [45]:
compute_mean_metrics({k:v for k,v in ood_metrics_decoded.items() if "10000" in k})

{'r_squared': 0.8533951642409745,
 'sinkhorn_div_1': 20.31417579650879,
 'sinkhorn_div_10': 6.025485992431641,
 'sinkhorn_div_100': 3.3319061279296873,
 'e_distance': 6.475952334054742,
 'mmd': 0.23551200519238286}

In [46]:
adata_pred_ood[adata_pred_ood.obs["perturbation"]=="Vehicle"].n_obs

3000

In [48]:
adata_test.obs.head()

,cell_type,dose,dose_character,dose_pattern,g1s_score,g2m_score,pathway,pathway_level_1,pathway_level_2,product_dose,...,cell_line,logdose,condition,pubchem_name,pubchem_ID,smiles,control,ood,is_ood,split
index,,,,,,,,,,,,,,,,,,,,,
A01_E09_RT_BC_100_Lig_BC_306-0-0,A549,10.0,10,4,0.000000,1.980748,DNA Damage,DNA damage & DNA repair,Nucleotide analog,Raltitrexed_10,...,A549,1.0,A549_Raltitrexed_10.0,Raltitrexed,135400182,CC1=NC2=C(C=C(C=C2)CN(C)C3=CC=C(S3)C(=O)NC(CCC...,0,not ood,False,test
A01_E09_RT_BC_101_Lig_BC_109-0-0,A549,0.0,0,3,0.000000,0.000000,Vehicle,Vehicle,Vehicle,Vehicle_0,...,A549,0.0,A549_Vehicle_0.0,NaN,NaN,NaN,1,not ood,False,test
A01_E09_RT_BC_101_Lig_BC_229-0-0,A549,10.0,10,4,1.817254,2.801225,Apoptosis,Protein folding & Protein degradation,E3 ubiquitin ligase activity,Lenalidomide (CC-5013)_10,...,A549,1.0,A549_Lenalidomide_(CC-5013)_10.0,Lenalidomide,216326,C1CC(=O)NC(=O)C1N2CC3=C(C2=O)C=CC=C3N,0,not ood,False,test
A01_E09_RT_BC_102_Lig_BC_265-0-0,A549,100.0,100,3,0.646621,1.315667,Protein Tyrosine Kinase,Tyrosine kinase signaling,RTK activity,Linifanib (ABT-869)_100,...,A549,2.0,A549_Linifanib_(ABT-869)_100.0,Linifanib,11485656,CC1=CC(=C(C=C1)F)NC(=O)NC2=CC=C(C=C2)C3=C4C(=C...,0,not ood,False,test
A01_E09_RT_BC_102_Lig_BC_379-0-0,A549,10000.0,10000,1,0.915929,3.055782,Protein Tyrosine Kinase,Tyrosine kinase signaling,RTK activity,Linifanib (ABT-869)_10000,...,A549,4.0,A549_Linifanib_(ABT-869)_10000.0,Linifanib,11485656,CC1=CC(=C(C=C1)F)NC(=O)NC2=CC=C(C=C2)C3=C4C(=C...,0,not ood,False,test


In [50]:
adata_test[adata_test.obs["pathway"]=="Vehicle"].obs["cell_line"].value_counts()

cell_line
A549    500
Name: count, dtype: int64

In [51]:

adata_ood[adata_ood.obs["pathway"]=="Vehicle"].obs["cell_line"].value_counts()

cell_line
A549    500
Name: count, dtype: int64

In [53]:
adata_train["Vehicle" in adata_train.obs["condition"]].obs["cell_line"].value_counts()

cell_line
A549    1
Name: count, dtype: int64

In [54]:
adata_train["Vehicle" in adata_train.obs["condition"]]

View of AnnData object with n_obs × n_vars = 1 × 2002
    obs: 'cell_type', 'dose', 'dose_character', 'dose_pattern', 'g1s_score', 'g2m_score', 'pathway', 'pathway_level_1', 'pathway_level_2', 'product_dose', 'product_name', 'proliferation_index', 'replicate', 'size_factor', 'target', 'vehicle', 'perturbation', 'drug', 'cell_line', 'logdose', 'condition', 'pubchem_name', 'pubchem_ID', 'smiles', 'control', 'ood', 'is_ood', 'split'
    var: 'id', 'num_cells_expressed-0-0', 'num_cells_expressed-1-0', 'num_cells_expressed-1', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p', 'pca', 'rank_genes_groups_cov_all'
    obsm: 'X_pca', 'cell_line_emb', 'ecfp', 'ecfp_cell_line', 'ecfp_cell_line_dose', 'ecfp_cell_line_dose_more_dose', 'ecfp_cell_line_logdose', 'ecfp_cell_line_logdose_more_dose'
    varm: 'PCs', 'X_train_mean'
    layers: 'X_log1p', 'centered_X'

In [58]:
adata_ood[adata_ood.obs["]].obs["cell_line"].unique()

['A549', 'MCF7', 'K562']
Categories (3, object): ['A549', 'K562', 'MCF7']

In [60]:
adata_ood.obs["condition"]

index
A01_E09_RT_BC_116_Lig_BC_193-0-0    A549_Alvespimycin_(17-DMAG)_HCl_10.0
A01_E09_RT_BC_175_Lig_BC_81-0-0     A549_Alvespimycin_(17-DMAG)_HCl_10.0
A01_E09_RT_BC_328_Lig_BC_18-0-0            A549_Dacinostat_(LAQ824)_10.0
A01_E09_RT_BC_357_Lig_BC_85-0-0          A549_Tanespimycin_(17-AAG)_10.0
A01_E09_RT_BC_380_Lig_BC_55-0-0            A549_Dacinostat_(LAQ824)_10.0
                                                    ...                 
H12_E09_RT_BC_137_Lig_BC_304-1                          K562_Vehicle_0.0
H12_E09_RT_BC_181_Lig_BC_329-1                          K562_Vehicle_0.0
H12_E09_RT_BC_18_Lig_BC_18-1                            K562_Vehicle_0.0
H12_F10_RT_BC_291_Lig_BC_126-1                          K562_Vehicle_0.0
H12_F10_RT_BC_75_Lig_BC_115-1                           K562_Vehicle_0.0
Name: condition, Length: 13317, dtype: category
Categories (40, object): ['A549_Alvespimycin_(17-DMAG)_HCl_10.0', 'A549_Belinostat_(PXD101)_100.0', 'A549_Dacinostat_(LAQ824)_10.0', 'A549_Tane

In [53]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex/results/biolord"

In [54]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, "biolord_split_ood_metrics_encoded.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, "biolord_split_ood_metrics_decoded.csv"))
#pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, "biolord_split_test_metrics_encoded.csv"))
#pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, "biolord_split_test_metrics_decoded.csv"))
#pd.DataFrame.from_dict(train_metrics_encoded).to_csv(os.path.join(output_dir, "biolord_split_train_metrics_encoded.csv"))
#pd.DataFrame.from_dict(train_metrics_decoded).to_csv(os.path.join(output_dir, "biolord_split_train_metrics_decoded.csv"))